In [1]:
import import_ipynb
import timeit
import time
from datetime import datetime
import os
import glob
from tqdm import tqdm

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from dataloader.dataloaderC import VideoDataset
from models.Video_C3D import C3D

/home/moon/miniconda3/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


importing Jupyter notebook from /home/moon/research/sitting-posture-recognition/dataloader/dataloaderC.ipynb
importing Jupyter notebook from /home/moon/research/sitting-posture-recognition/models/Video_C3D.ipynb


In [2]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print("Device being used:", device)

nEpochs = 10  # Number of epochs for training
resume_epoch = 0  # Default is 0, change if want to resume
useTest = True # See evolution of the test set when training
nTestInterval = 20 # Run on test set every nTestInterval epochs
snapshot = 10 # Store a model every snapshot epochs
lr = 1e-3 # Learning rate

dataset = 'sitting'

save_dir = '/home/moon/research/sitting-posture-recognition'
num_classes = 6
modelName = 'C3D'
saveName = modelName + '-' + dataset
pretrained = True
clip_len=30

Device being used: cuda:1


In [3]:
def train_model(dataset=dataset, num_classes=num_classes, lr=lr,
                num_epochs=nEpochs, save_epoch=snapshot, useTest=useTest, test_interval=nTestInterval, pretrained=pretrained):
    
    model = C3D(num_classes=num_classes, pretrained=pretrained)
    
    criterion = nn.CrossEntropyLoss()  # standard crossentropy loss for classification
    # optimizer = optim.Adam(model.parameters(), lr=lr)
    optimizer = optim.SGD(model.parameters(), lr=lr)
    # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10,
    #                                       gamma=0.1)  # the scheduler divides the lr by 10 every 10 epochs

    if resume_epoch == 0:
        print("Training {} from scratch...".format(modelName))

    model.to(device)
    
    print('model loaded')
    criterion.to(device)

    print('Training model on {} dataset...'.format(dataset))
    train_dataloader = DataLoader(VideoDataset(dataset=dataset, split='train',clip_len=clip_len), batch_size=6, shuffle=True)
    val_dataloader   = DataLoader(VideoDataset(dataset=dataset, split='val',  clip_len=clip_len), batch_size=6)
    test_dataloader  = DataLoader(VideoDataset(dataset=dataset, split='test', clip_len=clip_len), batch_size=6)

    trainval_loaders = {'train': train_dataloader, 'val': val_dataloader}
    trainval_sizes = {x: len(trainval_loaders[x].dataset) for x in ['train', 'val']}
    test_size = len(test_dataloader.dataset) # 1234

    for epoch in range(resume_epoch, num_epochs): # 0 ~ 100
        
        for phase in ['train', 'val']:
            start_time = timeit.default_timer()

            # reset the running loss and corrects
            running_loss = 0.0
            running_corrects = 0.0

            if phase == 'train':
                # scheduler.step() is to be called once every epoch during training
                # scheduler.step()
                model.train()
            else:
                model.eval()

            for inputs, labels in tqdm(trainval_loaders[phase]):
                inputs = inputs.to(device) 
                labels = labels.to(device) 
                
                optimizer.zero_grad()

                if phase == 'train':
                    outputs, corr = model(inputs)
                else:
                    with torch.no_grad():
                        outputs, corr = model(inputs)

                # probs = nn.Softmax(dim=1)(outputs)
                probs = outputs
                preds = torch.max(probs, 1)[1] # indicies

                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / trainval_sizes[phase]
            epoch_acc = running_corrects.double() / trainval_sizes[phase]
            
            print("[{}] Epoch: {}/{} Loss: {} Acc: {}".format(phase, epoch+1, nEpochs, epoch_loss, epoch_acc))
            stop_time = timeit.default_timer()
            print("Execution time: " + str(stop_time - start_time) + "\n")
            
            if epoch % save_epoch == (save_epoch - 1):
                torch.save({
                    'epoch': epoch + 1,
                    'state_dict': model.state_dict(),
                    'opt_dict': optimizer.state_dict(),
                }, os.path.join(save_dir, 'models', saveName + '_epoch-' + str(epoch) + '.pth'))
                print("Save model at {}\n".format(os.path.join(save_dir, 'models', saveName + '_epoch-' + str(epoch) + '.pth')))

        # Eval using Test set
        if useTest and epoch % test_interval == (test_interval - 1):
            model.eval()
            start_time = timeit.default_timer()

            running_loss = 0.0
            running_corrects = 0.0

            for inputs, labels in tqdm(test_dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                with torch.no_grad():
                    outputs = model(inputs)
                # probs = nn.Softmax(dim=1)(outputs)
                probs = outputs
                preds = torch.max(probs, 1)[1]
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / test_size
            epoch_acc = running_corrects.double() / test_size

            print("[test] Epoch: {}/{} Loss: {} Acc: {}".format(epoch+1, nEpochs, epoch_loss, epoch_acc))
            stop_time = timeit.default_timer()
            print("Execution time: " + str(stop_time - start_time) + "\n")

In [4]:
if __name__ == "__main__":
    train_model()

Loading Pretrained weights
Training C3D from scratch...
model loaded
Training model on sitting dataset...
Number of train videos: 193
Number of val videos: 32
Number of test videos: 32


100%|██████████| 33/33 [00:38<00:00,  1.17s/it]


[train] Epoch: 1/10 Loss: 2.837897399546569 Acc: 0.21761658031088082
Execution time: 38.51915006712079



100%|██████████| 6/6 [00:04<00:00,  1.30it/s]


[val] Epoch: 1/10 Loss: 3.928849029354751 Acc: 0.1875
Execution time: 4.603540454059839



...

[val] Epoch: 9/10 Loss: 1.1122903343784856 Acc: 0.65625
Execution time: 4.616324137896299



100%|██████████| 33/33 [00:38<00:00,  1.16s/it]


[train] Epoch: 10/10 Loss: 0.0029225195154856526 Acc: 1.0
Execution time: 38.21821031719446

Save model at /home/moon/research/sitting-posture-recognition/models/C3D-sitting_epoch-9.pth



100%|██████████| 6/6 [00:04<00:00,  1.29it/s]


[val] Epoch: 10/10 Loss: 1.0771007788862335 Acc: 0.65625
Execution time: 4.658220365643501

Save model at /home/moon/research/sitting-posture-recognition/models/C3D-sitting_epoch-9.pth

